# Step 1: Company Setup - Organization Context and Infrastructure

This notebook establishes the **company context** for cybersecurity investigations. It creates:

- **Company Identity**: Primary organization identity object
- **Employee Identities**: User accounts, emails, and identities for company personnel
- **IT System Identities**: Infrastructure and system components
- **Hardware Asset Identities**: Physical and virtual assets
- **Company Context Storage**: All objects organized in `/identity--{company-uuid}/` directory

## ✅ What This Notebook Does

1. **Initialize Company Context**: Create the company-specific context directory
2. **Create Company Identity**: Generate the primary organization identity
3. **Setup Company Users**: Create employee identity objects with categorized storage
4. **Configure IT Systems**: Create identity objects for infrastructure components
5. **Register Hardware Assets**: Create identity objects for physical/virtual assets
6. **Establish Relationships**: Link objects with appropriate STIX relationships

## 📁 Context Memory Structure Created

```
context_mem/
├── context_map.json                    # Global context routing
└── identity--{company-uuid}/           # Company-specific context
    ├── company.json                    # Company identity object
    ├── users.json                      # Employee identity objects
    ├── systems.json                    # IT system identity objects
    ├── assets.json                     # Hardware/asset identity objects
    └── edges.json                      # Company relationships
```

## 🎯 Prerequisites

- **User Setup Complete**: Must run `Step_0_User_Setup.ipynb` first
- **Company Data Available**: Input data files must exist for company, users, systems, and assets
- **Context Memory Initialized**: User context must be established

## A.1 Environment Setup

Initialize the same core libraries as user setup, ensuring consistent STIX object handling. This section:

- Loads STIX 2.1 libraries for company object creation
- Imports OS Threat extensions for enhanced organizational modeling
- Sets up logging for company context operations
- Ensures compatibility with existing user context

In [88]:
# Install required STIX libraries (if not already installed)
import sys
!{sys.executable} -m pip install stixorm

# Import core STIX 2.1 objects for cybersecurity intelligence
from stixorm.module.authorise import import_type_factory
from stixorm.module.definitions.stix21 import (
    Identity, EmailAddress, UserAccount, Relationship, Bundle
)

# Import OS Threat extensions for enhanced organizational modeling
from stixorm.module.definitions.os_threat import (
    IdentityContact, EmailContact, SocialMediaContact, ContactNumber
)

# Import processing utilities and result handling
from stixorm.module.typedb_lib.instructions import ResultStatus, Result
from stixorm.module.parsing import parse_objects

# Initialize import factory for STIX object creation
import_type = import_type_factory.get_all_imports()

# Setup logging for company context operations
import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

print("✅ STIX libraries initialized for company context")
print("✅ OS Threat organizational extensions loaded")
print("✅ Logging configured for company operations")

✅ STIX libraries initialized for company context
✅ OS Threat organizational extensions loaded
✅ Logging configured for company operations



[notice] A new release of pip is available: 24.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## A.2 Path Configuration

Configure Python paths and working directory for company context operations. This ensures:

- Access to Brett Blocks utility functions
- Proper relative path handling for company data files
- Consistent file system operations across environments

In [89]:
# Configure Python path for relative imports
import sys
sys.path.append('../')
import os

# Get current working directory for context operations
cwd = os.getcwd()

print(f"✅ Working directory: {cwd}")
print("✅ Python path configured for company context operations")

✅ Working directory: c:\projects\brett_blocks\Orchestration
✅ Python path configured for company context operations


## A.3 Import Company Context Utilities and Configure Data

Import specialized utility functions for company context management:

- **Company Context Creation**: `invoke_create_company_context()` for context initialization
- **Company Object Storage**: `invoke_save_company_context_block()` with category support
- **STIX Object Creation**: Identity, user account, and email utilities
- **Relationship Management**: Functions for linking company objects

Also configure the company and organizational data structures.

In [90]:
import json

# Import Brett Blocks utility functions for company context management
from Utilities.local_make_general import invoke_create_company_context
from Utilities.local_make_general import invoke_save_company_context_block
from Utilities.local_make_general import invoke_update_company_relations_block

# Import utilities for data processing and port emulation
from Utilities.util import emulate_ports, unwind_ports, conv

# Import STIX object creation utilities
from Utilities.local_make_sdo import invoke_make_identity_block
from Utilities.local_make_sco import invoke_make_user_account_block, invoke_make_email_addr_block
from Utilities.local_make_sro import invoke_sro_block

# Configure base paths for input data and results
path_base = "../Block_Families/StixORM/"
results_base = "../Orchestration/Results/"
sro_data = "SRO/Relationship/sro_attributed.json"

# Define company base configuration
company_base = {
    "ident": "SDO/Identity/identity_TR_user_company.json",
    "results": "step0/company"
}

# Define company user identities (employees)
# Note: paths are relative to the StixORM directory since utility functions add path_base
company_users_data = [
    {
        "who": "usr1",
        "email": "SCO/Email_Addr/email_addr_IT_user1.json",
        "acct": "SCO/User_Account/usr_account_IT_user1.json",
        "ident": "SDO/Identity/identity_IT_user1.json",
        "results": "step0/harry1"
    },
    {
        "who": "usr2",
        "email": "SCO/Email_Addr/email_addr_IT_user2.json",
        "acct": "SCO/User_Account/usr_account_IT_user2.json",
        "ident": "SDO/Identity/identity_IT_user2.json",
        "results": "step0/jane2"
    },
    {
        "who": "usr3",
        "email": "SCO/Email_Addr/email_addr_IT_user3.json",
        "acct": "SCO/User_Account/usr_account_IT_user3.json",
        "ident": "SDO/Identity/identity_IT_user3.json",
        "results": "step0/sally3"
    }
]

# Define IT systems configuration
systems_base = [
    {
        "data_path": "SDO/Identity/identity_Exchange.json",
        "results": "step0/system1"
    }
]

# Define hardware assets configuration
assets_base = [
    {
        "data_path": "SDO/Identity/identity_Laptop1.json",
        "results": "step0/asset1"
    },
    {
        "data_path": "SDO/Identity/identity_Laptop2.json",
        "results": "step0/asset2"
    },
    {
        "data_path": "SDO/Identity/identity_Laptop3.json",
        "results": "step0/asset3"
    }
]

print("✅ Company context utilities imported successfully")
print(f"✅ Base paths configured:")
print(f"   - Objects: {path_base}")
print(f"   - Results: {results_base}")
print(f"✅ Company configuration loaded:")
print(f"   - Company identity: {company_base['ident']}")
print(f"   - Company users: {len(company_users_data)} employees")
print(f"   - IT systems: {len(systems_base)} systems")
print(f"   - Hardware assets: {len(assets_base)} assets")

✅ Company context utilities imported successfully
✅ Base paths configured:
   - Objects: ../Block_Families/StixORM/
   - Results: ../Orchestration/Results/
✅ Company configuration loaded:
   - Company identity: SDO/Identity/identity_TR_user_company.json
   - Company users: 3 employees
   - IT systems: 1 systems
   - Hardware assets: 3 assets


## C. Create the Company Identity

Initialize the company context and create the primary organization identity. This process:

1. **Creates Company Context Directory**: Establishes `/identity--{company-uuid}/` directory structure
2. **Generates Company Identity**: Creates the primary organizational STIX Identity object
3. **Initializes Context Routing**: Updates global `context_map.json` for context routing
4. **Enables Multi-Tenant Storage**: Sets up category-specific storage for company objects

This company identity serves as the root organization object for all company-related cybersecurity activities.

In [91]:
# Extract company identity configuration paths
company_ident_path = company_base["ident"]
company_results_path = company_base["results"]

print(f"📝 Creating company identity from: {company_ident_path}")

# Create the company STIX Identity object
# This generates the primary organizational identity with company information
comp_ident = invoke_make_identity_block(company_ident_path, company_results_path)

# Configure context type for company context creation
context_type = {
    "context_type": "company"
}

# Define output paths for the company identity object
ident_results_obj_path = results_base + company_results_path + "__ident.json"
ident_results_context_path = results_base + "step0/context/company__ident_context.json"

# Initialize company context and save company identity
# This creates the /identity--{uuid}/ directory structure and stores the company identity
result = invoke_create_company_context(ident_results_obj_path, ident_results_context_path)

print(f"✅ Company context initialized successfully")
print(f"   - Company identity object: {ident_results_obj_path}")
print(f"   - Context storage: {ident_results_context_path}")
print(f"   - Context creation result: {result}")
print(f"🏢 Company identity is now active and ready for employee/asset registration")

📝 Creating company identity from: SDO/Identity/identity_TR_user_company.json
type identity->{'identity_form': {'base_required': {'type': 'identity', 'spec_version': '2.1', 'id': '', 'created': '', 'modified': ''}, 'base_optional': {'created_by_ref': '', 'revoked': None, 'labels': [], 'lang': '', 'external_references': [], 'object_marking_refs': [], 'granular_markings': [], 'defanged': None}, 'object': {'name': 'Example Company', 'description': 'An Example Organisation', 'identity_class': 'organization', 'sectors': 'technology'}, 'extensions': {}, 'sub': {}}}
type identity-><class 'dict'>, type email-><class 'NoneType'>, type user acct-><class 'NoneType'>
Step 1 >>
Step 2 >>
Step 3 >>
{
    "type": "identity",
    "spec_version": "2.1",
    "id": "identity--9a82c52d-d592-4f06-9894-88117f5c0b84",
    "created": "2025-10-30T05:24:48.879Z",
    "modified": "2025-10-30T05:24:48.879Z",
    "name": "Example Company",
    "description": "An Example Organisation",
    "identity_class": "organiz

## D. Create Company Users and Add to Context

Generate complete identity objects for all company employees and store them in the company context. For each employee, this creates:

1. **User Account Object**: Employee system account with login credentials
2. **Email Address Object**: Employee email linked to their account
3. **Identity Object**: Complete employee identity linking account and email
4. **Category Storage**: All objects stored in `users.json` with proper categorization

Company users are stored separately from personal team members to enable:
- **Role-Based Access Control**: Different permissions based on company roles
- **Incident Response**: Quick identification of affected employees
- **Organizational Modeling**: Understanding of company structure and relationships

In [92]:
# Process each company user (employee) identity configuration
employee_count = 0

for ident in company_users_data:
    results_paths = {}
    
    # Process only company user identities (those starting with "usr")
    if ident["who"][:3] == "usr":
        employee_count += 1
        
        # Extract object paths and results path for this employee
        email_path = ident["email"]
        acct_path = ident["acct"]
        ident_path = ident["ident"]
        results_path = ident["results"]
        
        print(f"\n👤 Processing employee {employee_count}: {ident['who']}")
        
        # Configure context type for company user storage
        context_type = {
            "context_type": "users"  # Store in users.json category
        }
        
        # Define context pathways for employee objects
        acct_results_obj_path = results_base + results_path + "__usr_acct.json"
        acct_results_context_path = results_base + "step0/context/usr_" + results_path[-1:] + "__usr_acct_context.json"
        email_results_obj_path = results_base + results_path + "__email.json"
        email_results_context_path = results_base + "step0/context/usr_" + results_path[-1:] + "__email.json"
        ident_results_obj_path = results_base + results_path + "__ident.json"
        ident_results_context_path = results_base + "step0/context/usr_" + results_path[-1:] + "__ident_context.json"
        
        # Create and save employee user account to company context
        user_acct = invoke_make_user_account_block(acct_path, results_path)
        result = invoke_save_company_context_block(acct_results_obj_path, acct_results_context_path, context_type)
        print(f"   ✅ User account created and saved to company context: {result}")
        
        # Create and save employee email address to company context
        user_email_addr = invoke_make_email_addr_block(email_path, results_path, user_acct)
        result = invoke_save_company_context_block(email_results_obj_path, email_results_context_path, context_type)
        print(f"   ✅ Email address created and saved to company context: {result}")
        
        # Create and save employee identity to company context
        user_ident = invoke_make_identity_block(ident_path, results_path, email_results=user_email_addr, acct_results=user_acct)
        result = invoke_save_company_context_block(ident_results_obj_path, ident_results_context_path, context_type)
        print(f"   ✅ Identity created and saved to company context: {result}")

print(f"\n🎉 Successfully created {employee_count} employee identities")
print(f"✅ All employees stored in company context (/identity--{{uuid}}/users.json)")
print(f"✅ Employee identities are now active for company operations")


👤 Processing employee 1: usr1
{
    "type": "user-account",
    "spec_version": "2.1",
    "id": "user-account--597ad4d4-35ba-585d-8f6d-134a75032f9b",
    "user_id": "79563902",
    "account_login": "nsmith",
    "account_type": "sales,",
    "display_name": "Naive Smith"
}
input data->{'stix_object': {'type': 'user-account', 'spec_version': '2.1', 'id': 'user-account--597ad4d4-35ba-585d-8f6d-134a75032f9b', 'user_id': '79563902', 'account_login': 'nsmith', 'account_type': 'sales,', 'display_name': 'Naive Smith'}, 'context_type': {'context_type': 'users'}}
from ports 
stix_object->{'type': 'user-account', 'spec_version': '2.1', 'id': 'user-account--597ad4d4-35ba-585d-8f6d-134a75032f9b', 'user_id': '79563902', 'account_login': 'nsmith', 'account_type': 'sales,', 'display_name': 'Naive Smith'}
context type->users
   ✅ User account created and saved to company context: {'context_result': 'Company identity--9a82c52d-d592-4f06-9894-88117f5c0b84\nOptions context saved -> users\nstix_id -> us

## E. Create and Save IT System Identities

Generate STIX Identity objects for all IT systems and infrastructure components. This creates:

- **System Identity Objects**: Each IT system represented as a STIX Identity
- **Infrastructure Mapping**: Complete inventory of company IT infrastructure
- **System Context Storage**: All systems stored in `systems.json` with proper categorization
- **Incident Readiness**: Systems available for threat attribution and impact analysis

IT systems include servers, networks, applications, databases, and other infrastructure components that may be involved in cybersecurity incidents.

In [93]:
# Process each IT system configuration
system_count = 0

for IT_system in systems_base:
    system_count += 1
    
    # Extract system identity path and results configuration
    system_ident_path = IT_system["data_path"]
    system_results_path = IT_system["results"]
    
    print(f"\n🖥️ Processing IT system {system_count}: {system_results_path}")
    
    # Configure context type for IT systems storage
    context_type = {
        "context_type": "systems"  # Store in systems.json category
    }
    
    # Create the IT system STIX Identity object
    # This represents the system as a cybersecurity entity for threat modeling
    system_ident = invoke_make_identity_block(system_ident_path, system_results_path)
    
    # Define output paths for the system identity object
    ident_results_obj_path = results_base + system_results_path + "__ident.json"
    ident_results_context_path = results_base + "step0/context/system" + system_results_path[-1:] + "__ident_context.json"
    
    # Save the system identity to company context
    # This adds the system to the company's IT infrastructure inventory
    result = invoke_save_company_context_block(ident_results_obj_path, ident_results_context_path, context_type)
    
    print(f"   ✅ IT system identity created and saved to company context: {result}")

print(f"\n🎉 Successfully created {system_count} IT system identities")
print(f"✅ All systems stored in company context (/identity--{{uuid}}/systems.json)")
print(f"✅ IT infrastructure inventory is now complete and ready for threat modeling")


🖥️ Processing IT system 1: step0/system1
type identity->{'identity_form': {'base_required': {'type': 'identity', 'spec_version': '2.1', 'id': '', 'created': '', 'modified': ''}, 'base_optional': {'created_by_ref': '', 'revoked': None, 'labels': [], 'lang': '', 'external_references': [{'source_name': 'Exchange Server', 'description': 'our email server', 'url': 'https://mail.echange.microsoft.com', 'external_id': 'admin'}], 'object_marking_refs': [], 'granular_markings': [], 'defanged': None}, 'object': {'name': 'Microsoft Exchange', 'description': 'My Local Exchange Server', 'roles': ['server'], 'identity_class': 'system', 'sectors': ['technology']}, 'extensions': {}, 'sub': {}}}
type identity-><class 'dict'>, type email-><class 'NoneType'>, type user acct-><class 'NoneType'>
Step 1 >>
Step 2 >>
Step 3 >>
{
    "type": "identity",
    "spec_version": "2.1",
    "id": "identity--b8e0b70a-3280-484b-9b47-b1e70dd87bf3",
    "created": "2025-10-30T05:24:50.200Z",
    "modified": "2025-10-30

## F. Create and Save IT Hardware Asset Identities

Generate STIX Identity objects for all hardware assets and physical infrastructure. This creates:

- **Asset Identity Objects**: Each hardware asset represented as a STIX Identity
- **Physical Infrastructure Mapping**: Complete inventory of company hardware assets
- **Asset Context Storage**: All assets stored in `assets.json` with proper categorization
- **Physical Security Modeling**: Assets available for physical threat analysis

Hardware assets include servers, workstations, network equipment, mobile devices, IoT devices, and other physical components that may be targets or involved in cybersecurity incidents.

In [94]:
# Process each hardware asset configuration
asset_count = 0

for i, asset in enumerate(assets_base):
    asset_count += 1
    
    # Extract asset identity path and results configuration
    asset_ident_path = asset["data_path"]
    asset_results_path = asset["results"]
    
    print(f"\n🔧 Processing hardware asset {asset_count}: {asset_results_path}")
    
    # Configure context type for hardware assets storage
    context_type = {
        "context_type": "assets"  # Store in assets.json category
    }
    
    # Create the hardware asset STIX Identity object
    # This represents the asset as a cybersecurity entity for threat modeling
    asset_ident_list = invoke_make_identity_block(asset_ident_path, asset_results_path)
    
    # Define output paths for the asset identity object
    ident_results_obj_path = results_base + asset_results_path + "__ident.json"
    ident_results_context_path = results_base + "step0/context/hardware" + asset_results_path[-1:] + str(i) + "__ident_context.json"
    
    # Save the asset identity to company context
    # This adds the asset to the company's hardware inventory
    result = invoke_save_company_context_block(ident_results_obj_path, ident_results_context_path, context_type)
    
    print(f"   ✅ Hardware asset identity created and saved to company context: {result}")

print(f"\n🎉 Successfully created {asset_count} hardware asset identities")
print(f"✅ All assets stored in company context (/identity--{{uuid}}/assets.json)")
print(f"✅ Hardware asset inventory is now complete and ready for security modeling")


🔧 Processing hardware asset 1: step0/asset1
type identity->{'identity_form': {'base_required': {'type': 'identity', 'spec_version': '2.1', 'id': '', 'created': '', 'modified': ''}, 'base_optional': {'created_by_ref': '', 'revoked': None, 'labels': [], 'lang': '', 'external_references': [{'source_name': 'HP Ultima 10', 'description': '16GB RAM, 1TB SD, Microsoft Windows 11 Professional', 'url': 'https://mail.echange.microsoft.com', 'external_id': 'Asset--629456a'}], 'object_marking_refs': [], 'granular_markings': [], 'defanged': None}, 'object': {'name': 'HP Laptop 1', 'description': "Usr 1's Laptop", 'roles': ['laptop'], 'identity_class': 'asset', 'sectors': ['technology']}, 'extensions': {}, 'sub': {}}}
type identity-><class 'dict'>, type email-><class 'NoneType'>, type user acct-><class 'NoneType'>
Step 1 >>
Step 2 >>
Step 3 >>
{
    "type": "identity",
    "spec_version": "2.1",
    "id": "identity--d5318d91-91f8-4675-ae0e-d58e1195ce58",
    "created": "2025-10-30T05:24:50.273Z",
 

## G. Create and Save Organizational Relationships

Establish STIX Relationship Objects (SROs) that link company entities together. This creates:

- **Employment Relationships**: Link employees to the company
- **Asset Ownership**: Link hardware assets to the company
- **System Administration**: Link IT systems to responsible employees
- **Organizational Hierarchy**: Model reporting structures and dependencies

These relationships enable sophisticated threat modeling, impact analysis, and incident response by understanding how company entities are connected.

In [95]:
# Define the relationship configuration path and results location
stix_object_path = "../Block_Families/Context/Update_context/relationship_type.json"
results_path = results_base + "step0/_relations_made.json"

print(f"🔗 Creating organizational relationships...")
print(f"   - Configuration: {stix_object_path}")
print(f"   - Results: {results_path}")

# Create and save organizational relationships
# This function processes relationship definitions and creates appropriate SRO objects
# Common relationships include:
#   - employed-by: employees to company
#   - asset-of: hardware assets to company  
#   - system-of: IT systems to company
#   - administers: employees to systems/assets

# Note: Relationship creation is currently commented out for debugging
# Uncomment the following lines when relationship configuration is ready:

# result_string = invoke_update_company_relations_block(stix_object_path, results_path)
# print(f"✅ Organizational relationships created: {result_string}")

print("⚠️ Relationship creation currently disabled for debugging")
print("✅ Company context structure is complete without relationships")
print("🔧 Enable relationship creation when configuration is validated")

🔗 Creating organizational relationships...
   - Configuration: ../Block_Families/Context/Update_context/relationship_type.json
   - Results: ../Orchestration/Results/step0/_relations_made.json
⚠️ Relationship creation currently disabled for debugging
✅ Company context structure is complete without relationships
🔧 Enable relationship creation when configuration is validated


## 🎉 Company Setup Complete!

Your company context has been successfully established with:

### ✅ Created Company Infrastructure
- **Company Identity**: Primary organization identity object
- **Employee Identities**: Complete user accounts, emails, and identities for all personnel
- **IT System Identities**: Infrastructure components ready for threat modeling
- **Hardware Asset Identities**: Physical assets inventory for security analysis
- **Company Context Storage**: All objects organized in categorized company context

### 📁 Context Memory Structure
```
context_mem/
├── context_map.json                    # Global context routing
├── usr/                               # Personal user context (from Step 0)
│   ├── cache_me.json                  # Your personal identity
│   └── cache_team.json                # Team member identities
└── identity--{company-uuid}/           # Company context
    ├── company.json                   # Company identity object
    ├── users.json                     # Employee identity objects
    ├── systems.json                   # IT system identity objects
    ├── assets.json                    # Hardware asset identity objects
    └── edges.json                     # Company relationships
```

### 🚀 Next Steps
You're now ready to:
1. **Create Security Incidents**: Use company context for incident investigations
2. **Perform Threat Modeling**: Analyze threats against company infrastructure
3. **Conduct Impact Analysis**: Assess incident effects on employees, systems, and assets
4. **Execute Response Plans**: Coordinate incident response using organizational structure
5. **Generate Threat Intelligence**: Create STIX objects for threat sharing and analysis

### 🏢 Company Context Features
- **Multi-Tenant Support**: Company context isolated from other organizations
- **Category-Based Storage**: Objects organized by type (users, systems, assets)
- **Relationship Modeling**: Foundation for organizational relationship mapping
- **Incident Readiness**: Complete organizational inventory for cybersecurity operations

Your company context will persist across all Brett Blocks sessions and provides the comprehensive organizational foundation for advanced cybersecurity intelligence operations.